In [280]:
import folium
import numpy as np
import pandas as pd
import networkx as nx

---
# Data Import
---

### Read the airlines data with all the flights from src to dest

In [281]:
airlines_data = pd.read_csv("data/airlines.csv")
airlines_data = airlines_data[["src_name", "src_id", "dest_name", "dest_id"]]
airlines_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67663 entries, 0 to 67662
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   src_name   67663 non-null  object
 1   src_id     67663 non-null  object
 2   dest_name  67663 non-null  object
 3   dest_id    67663 non-null  object
dtypes: object(4)
memory usage: 2.1+ MB


### Read aiports data with corresponding latitude, longitude and altitude

In [282]:
airports_data = pd.read_csv("data/airports.csv")
airports_data = airports_data[["id", "name", "latitude", "longitude"]]
airports_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12668 entries, 0 to 12667
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         12668 non-null  int64  
 1   name       12668 non-null  object 
 2   latitude   12668 non-null  float64
 3   longitude  12668 non-null  float64
dtypes: float64(2), int64(1), object(1)
memory usage: 396.0+ KB


---
# Data Processing and Cleaning
---

### Fill the "\N" values of the airlines dataset and cast object columns to int64

In [283]:
name_to_id_mapping = pd.Series(airports_data["id"].values, index=airports_data["name"]).to_dict()
airlines_data.replace({"\\N": pd.NA}, inplace=True)
airlines_data["src_id"] = airlines_data["src_id"].fillna(airlines_data["src_name"].map(name_to_id_mapping))
airlines_data["dest_id"] = airlines_data["dest_id"].fillna(airlines_data["dest_name"].map(name_to_id_mapping))
airlines_data.dropna(inplace=True)
airlines_data["src_id"] = airlines_data["src_id"].astype(np.int64)
airlines_data["dest_id"] = airlines_data["dest_id"].astype(np.int64)
airlines_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 67572 entries, 0 to 67662
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   src_name   67572 non-null  object
 1   src_id     67572 non-null  int64 
 2   dest_name  67572 non-null  object
 3   dest_id    67572 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 2.6+ MB


### Merge the datasets and get the latitude and longitude for both source and destination airports

In [284]:
data = pd.merge(airlines_data, airports_data, left_on=["src_name"], right_on=["name"], how="left")
data = data.rename(columns={"latitude" : "src_latitude", "longitude": "src_longitude"})[["src_id", "src_name", "src_latitude", "src_longitude", "dest_id", "dest_name"]]
data = pd.merge(data, airports_data, left_on=["dest_name"], right_on=["name"], how="left")
data = data.rename(columns={"latitude" : "dest_latitude", "longitude": "dest_longitude"})[["src_id", "src_name", "src_latitude", "src_longitude", "dest_id", "dest_name", "dest_latitude", "dest_longitude"]]
data.dropna(inplace=True)
data

,src_id,src_name,src_latitude,src_longitude,dest_id,dest_name,dest_latitude,dest_longitude
0,2965,AER,43.449902,39.956600,2990,KZN,55.606201,49.278702
1,2966,ASF,46.283298,48.006302,2990,KZN,55.606201,49.278702
2,2966,ASF,46.283298,48.006302,2962,MRV,44.225101,43.081902
3,2968,CEK,55.305801,61.503300,2990,KZN,55.606201,49.278702
4,2968,CEK,55.305801,61.503300,4078,OVB,55.012600,82.650703
...,...,...,...,...,...,...,...,...
67567,6334,WYA,-33.058899,137.514008,3341,ADL,-34.945000,138.531006
67568,4029,DME,55.408798,37.906300,2912,FRU,43.061298,74.477600
67569,2912,FRU,43.061298,74.477600,4029,DME,55.408798,37.906300
67570,2912,FRU,43.061298,74.477600,2913,OSS,40.609001,72.793297


---
# Data Analysis (Part I)
---

### Counting the occurrences of each source name

In [285]:
src_count = data["src_name"].value_counts().reset_index()
src_count.columns = ["src_name", "count"]
src_count

,src_name,count
0,ATL,915
1,ORD,558
2,LHR,527
3,PEK,527
4,CDG,524
...,...,...
3351,OGS,1
3352,OLF,1
3353,YGZ,1
3354,PVC,1


### Counting the occurrences of each destination name

In [286]:
dest_count = data["dest_name"].value_counts().reset_index()
dest_count.columns = ["dest_name", "count"]
dest_count

,dest_name,count
0,ATL,911
1,ORD,550
2,PEK,527
3,LHR,524
4,CDG,517
...,...,...
3359,OER,1
3360,ESL,1
3361,OGL,1
3362,PES,1


---
# Visualization (Part I)
---

### Create a function to visualiza data in a map

In [287]:
def create_visualization(df, filename):
    vis = folium.Map(location=[0,0], zoom_start=3, tiles="cartodb dark_matter")
    for _, route in df.iterrows():
        folium.PolyLine([(route["src_latitude"], route["src_longitude"]), (route["dest_latitude"], route["dest_longitude"])], color="cyan", weight=0.1, opacity=0.5).add_to(vis)
    vis.save(filename)

### Visualize all the airlines data in one map

In [288]:
create_visualization(data, "visualization_part_1.html")

---
# Node Analysis (Part I)
---

### Create the

In [289]:
G = nx.Graph()

for _, row in data.iterrows():
    G.add_edge(row["src_name"], row["dest_name"])